In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import seml.database as db_utils

import jkutils.mpl_latex_template as mplt

import os 
os.environ["PATH"] += os.pathsep + '/nfs/homedirs/schuchaj/texlive/2020/bin/x86_64-linux'

/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [2]:
collection = 'cert_collective'
collection_train = 'train_collective'

jk_config = {
    'username': 'schuchaj',
    'password': 'LBGqsHEY',
    'host': 'fs.kdd.in.tum.de',
    'port': 27017,
    'db_name': 'schuchaj'
}

col = db_utils.get_collection(collection, mongodb_config=jk_config)
col_train = db_utils.get_collection(collection_train, mongodb_config=jk_config)

In [3]:
def find_items(dataset, model, max_rad, local_budget_descriptor, pf_minus_adj, pf_plus_att, pf_minus_att, num_attackers=None, seed=None):
    
    db_filter = {
        'config.restrictions.dataset' : dataset,
        'config.restrictions.model' : model,
        'config.max_rad' : max_rad,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj,
        'config.pf_plus_att': pf_plus_att,
        'config.pf_minus_att': pf_minus_att,
        'config.local_budget_descriptor': local_budget_descriptor,
        'config.num_attackers': num_attackers
    }
    if seed is not None:
        db_filter['config.seed'] = seed
    if col.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col.find(db_filter)
    
    ret = []
    
    for exp in exps:
        config = exp['config']
        cert_file_dir = exp['result']['cert_file']
        try:
            cert = torch.load(cert_file_dir)
        except:
            cert = torch.load(f'/nfs/students/schuchaj/sparse_smoothing_results/cert_collective/{cert_file_dir.split("/")[-1]}')

        ret.append((exp, config, cert))
            
    return ret

In [4]:
def grid_from_exp(exp, collective=True):
    cert = exp[2]
    if collective:
        return cert['collective_grid']
    else:
        idx_test = cert['idx_test']
        return ((cert['grid_base'][idx_test] > 0.5).sum(axis=0))

In [5]:
# Styling
sns.set()
pal = sns.color_palette('Blues_r', 7)

In [6]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)


budget_dict = {}
budget_dict_std = {}

for num_attackers in [4, 8, 16, 32, 64]:
    exps_collective_cora = find_items('cora_ml', 'GCN', [0, 0, 1000], [0, 0, 8], 0.0, 0.002, 0.6, num_attackers=num_attackers)
    grids_cora = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_cora])
    budget_dict[num_attackers] = np.mean(grids_cora, axis=0)
    budget_dict_std[num_attackers] = np.std(grids_cora, axis=0)

for i, n in enumerate([4, 8, 16, 32, 64]):
    plt.plot(np.arange(101) * 10,budget_dict[n], label=f'{n}', color=pal[i], clip_on=False, zorder=3)
    
legend = plt.legend(title='Attacker Nodes')
plt.xlabel('Attribute deletions')
plt.ylabel('Certified ratio')
plt.ylim(-0.03, 1.05)

plt.setp(legend.get_title(),fontsize='x-small')


plt.xlim(0, 1000)
plt.ylim(0, 1)

mplt.savefig(f'/nfs/homedirs/schuchaj/plotting/figures/experiments/num_attackers/attr_deletions', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})


Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello


In [7]:
pal = sns.color_palette('Blues_r', 8)

In [8]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)


budget_dict = {}
budget_dict_std = {}

for num_attackers in [4, 8, 16, 32, 64]:
    exps_collective_cora = find_items('cora_ml', 'GCN', [1000, 0, 0], [8, 0, 0], 0.4, 0.00, 0, num_attackers=num_attackers)
    grids_cora = np.vstack([grid_from_exp(exp)[:, 0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_cora])
    budget_dict[num_attackers] = np.mean(grids_cora, axis=0)
    budget_dict_std[num_attackers] = np.std(grids_cora, axis=0)

for i, n in enumerate([4, 8, 16, 32, 64]):
    plt.plot(np.arange(101) * 10,budget_dict[n], label=f'{n}', color=pal[i], clip_on=False, zorder=3)
    
legend = plt.legend(title='Att. nodes', loc='upper right')
plt.xlabel('Edge deletions')
plt.ylabel('Certified ratio')
plt.ylim(-0.03, 1.05)

plt.setp(legend.get_title(),fontsize='medium')

plt.xlim(0, 1000)
plt.ylim(0, 1)

mplt.savefig(f'/nfs/homedirs/schuchaj/plotting/figures/experiments/num_attackers/edge_deletions', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})


/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello


In [9]:
1000 / 16

62.5

In [10]:
# attackers:
4, 8, 16, 32, 64

(4, 8, 16, 32, 64)

In [11]:
64 * 16

1024